In [1]:
!git clone https://github.com/pie3636/newsjam.git
!mv newsjam/* .

Cloning into 'newsjam'...
remote: Enumerating objects: 439, done.
remote: Counting objects: 100% (439/439), done.
remote: Compressing objects: 100% (319/319), done.
remote: Total 439 (delta 236), reused 264 (delta 116), pack-reused 0
Receiving objects: 100% (439/439), 10.37 MiB | 5.34 MiB/s, done.
Resolving deltas: 100% (236/236), done.


Imports (Run Once)

In [2]:
!pip install selenium --upgrade;
!apt install chromium-chromedriver;
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install tweepy --upgrade;
!pip install python-dotenv --upgrade;

     |████████████████████████████████| 958 kB 19.6 MB/s 
     |████████████████████████████████| 356 kB 65.3 MB/s 
     |████████████████████████████████| 138 kB 61.2 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 3.6 MB 53.7 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following add

In [3]:
!python -m pip install -r requirements.txt;
!python -m spacy download fr_core_news_sm;

     |████████████████████████████████| 60 kB 5.2 MB/s 
     |████████████████████████████████| 306 kB 65.9 MB/s 
     |████████████████████████████████| 1.2 MB 28.1 MB/s 
     |████████████████████████████████| 3.4 MB 53.6 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 61 kB 435 kB/s 
     |████████████████████████████████| 132 kB 59.2 MB/s 
     |████████████████████████████████| 1.1 MB 39.6 MB/s 
     |████████████████████████████████| 243 kB 54.0 MB/s 
     |████████████████████████████████| 160 kB 81.5 MB/s 
     |████████████████████████████████| 192 kB 66.9 MB/s 
     |████████████████████████████████| 271 kB 62.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 14.

Twitter Setup and Authentication

In [1]:
import nltk
nltk.download('stopwords')

import os
from dotenv import load_dotenv
from summ.lsa import LSASummarizer
from summ.bert_embed import BertEmbeddingsSummarizer as Bert
from data import scraper_functions
from classification import log_reg_classifier as lrc
import tweepy
import numpy as np

#this will not work if you don't have the .env file with the API keys

load_dotenv('/content/API.env')
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
bearer = os.getenv('bearer')
access_key = os.getenv('access_key')
access_key_secret = os.getenv('access_key_secret')

client = tweepy.Client(bearer_token=bearer, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_key, access_token_secret=access_key_secret)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[LibLinear]

In [15]:
url_list = scraper_functions.actu_autoscraper('https://actu.fr/le-journal-de-vitre/societe/coronavirus', url_amount=10)
lsa_summ = LSASummarizer()
bert = Bert('flaubert/flaubert_large_cased')

for x in range(2,len(url_list)):
  text, url = scraper_functions.actu_scraper(url_list[x])
  print(url_list[x])
  print(text)
  lrc_results = lrc.classifier(text)
  if lrc_results == False:
    continue
  else:
    summary = lsa_summ.get_summary(text)
    print(summary)
    summary = summary[0]
    tweet_response = client.create_tweet(text=summary)
    client.create_tweet(text=url, in_reply_to_tweet_id=tweet_response.data['id']) #reply to the posted tweet immediately with the original URL
    url_list = []
    text = ''
    url = ''
    break

Some weights of the model checkpoint at flaubert/flaubert_large_cased were not used when initializing FlaubertModel: ['pred_layer.proj.bias', 'pred_layer.proj.weight']
- This IS expected if you are initializing FlaubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


https://actu.fr/bretagne/servon-sur-vilaine_35327/covid-19-pres-de-rennes-une-ecole-ferme-en-raison-dun-trop-grand-nombre-de-cas_47080657.html
Actu Bretagne Ille-et-Vilaine Servon-sur-Vilaine Covid-19 : près de Rennes, une école ferme en raison d'un trop grand nombre de cas  Le Journal de Vitré Mon actu Suivre Depuis plusieurs semaines, l’école Sainte-Marie de Servon-sur-Vilaine doit faire face à une multiplication de tests positifs au Covid-19 de ses élèves. Les écoliers de certaines classes ont déjà dû être isolés et des classes ont fermé ces derniers jours. Cette fois, ce mercredi 8 décembre 2021, l’établissement a appris qu’il devait immédiatement fermer ses portes, par arrêté préfectoral. À lire aussi Covid-19 : moins de classes fermées, mais plus de 1300 élèves positifs en une semaine en Bretagne  Pour une semaine Tous les élèves doivent donc être isolés pendant une semaine à compter de ce mercredi, soit jusqu’au mercredi 15 décembre inclus. 
('À lire aussi Covid-19 : moins de cl